In [1]:
import pandas as pd

In [2]:
options = "ABCD"
opt_cols = [f"Probability of {opt}" for opt in options]
pdf = pd.read_csv("players.csv") # player DataFrame
players = dict(pdf.set_index("Email Address")["What do you want your display name to be?"])
gdf = pd.read_csv("guesses.csv") # guess DataFrame
adf = pd.read_csv("answers.csv").set_index("qnum") # answer DataFrame
adf["correct_letter"] = adf["correct"].apply(lambda s: s[:1])

In [3]:
def brier(probs, idx_correct):
    tot = sum(probs)
    if tot == 0:
        return 2
    probs = [p/tot for p in probs]
    score = 0
    for i in range(len(probs)):
        expected = int(i == idx_correct) # 1 or 0
        score += (probs[i] - expected) ** 2
    return score

print(brier([100,0,0,0], 0))
print(brier([100,0,0,0], 3))
print(brier([0.1,0.1,0.1,0.7], 3))
print(brier([0.1,0.1,0.1,0.7], 1))
print(brier([0.15,0.15,0.15,0.55], 3))
print(brier([0.15,0.15,0.15,0.55], 1))

0.0
2.0
0.12000000000000002
1.32
0.26999999999999996
1.0699999999999998


In [7]:
def row_brier(row):
    qnum = row["Question Number"]
    if not qnum in adf.index:
        return 2
    correct = adf.loc[qnum, "correct_letter"]
    return brier(list(row[opt_cols]), options.index(correct))

for i in range(len(gdf)):
    gdf.loc[i, "brier"] = row_brier(gdf.loc[i])

In [9]:
def score_user(email, start, end):
    df = gdf[gdf["Email Address"] == email]
    df = df.drop_duplicates(subset="Question Number", keep="first")
    df = df.set_index("Question Number", drop=False)
    score = 0
    for qnum in range(start, end+1):
        score += df.loc[qnum, "brier"] if qnum in df.index else 2
    return score

def score_users(start, end):
    rows = []
    for email, alias in players.items():
        rows.append({"player":alias, "brier":score_user(email, start, end)})
    return pd.DataFrame(rows).sort_values(by="brier")
score_users(1, 3)

,player,brier
0,Tyler,1.745
